코드 출처  
[Python 동시 프로그래밍](https://nachwon.github.io/asyncio-futures/)

### 순차적 프로그래밍

In [1]:
import requests
import time

def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jyth

### 동시성 프로그래밍

requests.Session()은 threadsafe하지 않기 때문에 threading.local()을 통해 각 쓰레드마다 별도 객체를 만들어줘야함. 

In [2]:
import concurrent.futures
import requests
import threading
import time


thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10267 from https://www.jython.orgRead 10267 from https://www.jython.org

Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10267 from https://www.jython.org
Read 10267 from https://www.jython.org
Read 276 from

### 비동기 프로그래밍

1. asyncio.ensure_future()로 태스크 목록 생성
2. asyncio.gather()로 대기열 안의 task가 모두 끝날 때까지 session 사용
3. task가 모두 끝나면 await로 이벤트 루프에 제어권 반납

In [3]:
import asyncio
import time
import aiohttp


async def download_site(session, url):
    async with session.get(url) as response:
        print("Read {0} from {1}".format(response.content_length, url))


async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(download_all_sites(sites))
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration} seconds")

Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 from https://www.jython.org
Read 3549 fr

### 병렬성 프로그래밍

1. set_global_session에서 얻은 리턴값을 download_site에 못 넘겨주기 때문에 전역변수 session 생성
2. multiprocessing.Pool로 프로세서 개수에 맞는 프로세스 생성
3. 각 프로세스가 download_site가 sties의 항목에 매핑

In [4]:
import time
# jupyter mutliprocessing 에러 때문에 defs.py로 따로 저장 후 호출
import defs

"""
session = None


def set_global_session():
    global session
    if not session:
        session = requests.Session()


def download_site(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        print(f"{name}:Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)
"""

if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    defs.download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Downloaded 160 in 3.0816948413848877 seconds
